# Library 

In [1]:
pip install transformers==4.3.0

     |████████████████████████████████| 1.8 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 20.1 MB/s 
     |████████████████████████████████| 895 kB 49.2 MB/s 


In [2]:
pip install h5py==2.10.0

     |████████████████████████████████| 2.9 MB 4.2 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [3]:
pip install pyvi

     |████████████████████████████████| 8.5 MB 3.7 MB/s 
     |████████████████████████████████| 743 kB 29.8 MB/s 


In [4]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


# Load model and support functions

In [5]:
import re 

from pyvi.ViTokenizer import ViTokenizer

STOPWORDS = 'drive/My Drive/CODE/ViSocial/vietnamese-stopwords-dash.txt'
with open(STOPWORDS, "r") as ins:
    stopwords = []
    for line in ins:
        dd = line.strip('\n')
        stopwords.append(dd)
    stopwords = set(stopwords)

def filter_stop_words(train_sentences, stop_words):
    new_sent = [word for word in train_sentences.split() if word not in stop_words]
    train_sentences = ' '.join(new_sent)
        
    return train_sentences

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def preprocess(text, tokenized = True, lowercased = True):
    text = ViTokenizer.tokenize(text) if tokenized else text
    # text = filter_stop_words(text, stopwords)
    # text = deEmojify(text)
    text = text.lower() if lowercased else text
    return text

In [46]:
import pickle
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification

from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

phobert = AutoModelForSequenceClassification.from_pretrained("drive/MyDrive/CODE/ViSocial/transformer_model/victsd-toxic/phobert-v1-no-pp/", num_labels = 2)
tokenizer_phobert = AutoTokenizer.from_pretrained("vinai/phobert-base",use_fast=False)


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [38]:
import numpy as np

def sigmoid_array(x): 
    return 1 / (1 + np.exp(-x))

def predict(text, lowercased=False, tokenized=False):
    labels = {
        0: "NON-TOXIC",
        1: "TOXIC",
    }
    p_text = preprocess(text, lowercased=lowercased, tokenized=tokenized)

    X2 = tokenizer_phobert([p_text], truncation=True, padding=True, max_length=100)
    X2 = BuildDataset(X2, [1])
    y2 = Trainer(model=phobert).predict(X2).predictions
    y2 = sigmoid_array(y2)

    return labels[np.argmax(y2, axis=-1)[0]]

# Test

## Example 

In [50]:
print(predict("Vụ này sao có nhiều uẩn khúc. Kế hoạch cồng kềnh và đi quãn đường khá xa nhà. Nếu lỡ ng nhà đến đón đc em trc thì việc này là ko thành. Đâu lý nào bộc phát ý định tự tử trong 1 2 ngày đc"))

NON-TOXIC


In [53]:
print(predict("Giá như có ai đó đưa e đi thì đã ko đến nỗi, thương e quá, tại sao e ko nghĩ đến việc sắp đc gặp bạn bè, thầy cô nhỉ, lúc chưa điều tra ra thì mình cứ nghĩ ai đó đã hại e và căm phẫn kẻ đó, hôm nay bit thế này thì mình cứ nghẹn ở cổ vì thương xót cho e. Nói đến đây mình lại nhớ tới có lần e gái mình gặp chuyện buồn nên cũng thuê khách sạn nghỉ và chủ thấy biểu hiện buồn nên hỏi han thì cũng vơi bớt, . giá Như chủ nhà thấy e còn trẻ mà ở quê ra phố một mình ..."))

NON-TOXIC


In [54]:
print(predict("Lừa đảo giết người"))

TOXIC


## Manually inupt

In [ ]:
a = input()
print(predict_ensembles(a))

csgt rách 


CLEAN
